## 2. Preparing for Model Validation

#### 2.1 Calculating Regression Coefficients for all features

I will examine the coefficients and conduct a variable importance analysis to refine the feature coefficients. This was crucial in identifying which features were most impactful in predicting customer lifetime value (CLV). Despite these efforts, if the revised model exhibits high error rates, my next step will be to consider simplifying the calculation of customer lifespan (CLS). Instead of using weighted coefficients, I might revert to a more straightforward approach of multiplying [N_FirstRecency] by [N_YearlyFrequency]. This change would be based on the hypothesis that the inclusion of complex coefficients might be introducing unnecessary noise or distortion in the model, detracting from its predictive accuracy.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('hotel_cluster_normalized.csv')

# Split the data
X = df.drop(['CUSTOMER_ID', 'N_TotalRevenue'], axis=1)
y = df['N_TotalRevenue']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Coefficients
coefficients = model.coef_
feature_importance = pd.DataFrame(coefficients, index=X.columns, columns=['Coefficient']).abs()
feature_importance.sort_values(by='Coefficient', ascending=False, inplace=True)
print(feature_importance)


                                              Coefficient
N_FirstRecency                               1.606693e+09
N_Recency                                    1.279043e+09
N_LifetimeRecency                            1.182764e+09
Revenue_Segment_No Ancillary Spenders        3.729609e+08
Revenue_Segment_Low Ancillary Spenders       3.447274e+08
Revenue_Segment_Moderate Ancillary Spenders  2.730909e+08
Revenue_Segment_High Ancillary Spenders      1.531436e+08
Segment_No Ancillary Spenders                7.282971e+07
Segment_Low Ancillary Spenders               6.731529e+07
Segment_Moderate Ancillary Spenders          5.332184e+07
Segment_High Ancillary Spenders              2.994423e+07
N_FMonthlyBiasSD                             6.458193e-02
N_MMonthlyBiasSD                             6.345979e-02
N_FQuarterlyBiasSD                           1.181928e-02
N_MQuarterlyBiasSD                           1.172094e-02
N_Monetary                                   7.343400e-03
N_RecentMoneta

#### 2.2 Testing the Revised Model for CLS Calculation with Coefficients (Post-Feature Importance Analysis) and Focusing on Customer Activity in 2021
##### In this phase, I tested a modified version of the CLS (Customer Lifespan) calculation model, incorporating coefficients derived from a feature importance analysis. This approach aimed to enhance the model's accuracy by weighting the most influential factors more heavily in the CLS computation.
##### Additionally, the scope of the analysis was narrowed to customers active in both 2021 and the preceding years. By focusing on this specific customer segment, the model's effectiveness in capturing consistent customer behavior over time was assessed. This approach provided insights into the reliability and predictability of the CLV (Customer Lifetime Value) model for customers with a demonstrated history of engagement.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the original and normalized datasets
original_data = pd.read_csv('COLUMBIA_CAPSTONE_1M_FINAL_DATASET.csv', sep="|", header=None)
normalized_data = pd.read_csv('/content/hotel_cluster_normalized.csv')
original_data.columns = ['CUSTOMER_ID', 'CALENDAR_YEAR', 'CALENDAR_MONTH', 'PRODUCT_CATEGORY',
                         'LATEST_TRANSACTION_DATE', 'NUM_TRANSACTIONS', 'TOTAL_SPEND', 'MARRIOTT_REVENUE']

# Aggregate the original data by customer and year
yearly_data = original_data.groupby(['CUSTOMER_ID', 'CALENDAR_YEAR']).agg({
    'NUM_TRANSACTIONS': 'sum',
    'TOTAL_SPEND': 'sum'
}).reset_index()

# Identify customers with transactions in 2021 and before 2021
customers_2021 = set(yearly_data[yearly_data['CALENDAR_YEAR'] == 2021]['CUSTOMER_ID'])
customers_before_2021 = set(yearly_data[yearly_data['CALENDAR_YEAR'] < 2021]['CUSTOMER_ID'])
active_customers = customers_2021.intersection(customers_before_2021)

# Filter the yearly_data to include only these customers
filtered_yearly_data = yearly_data[yearly_data['CUSTOMER_ID'].isin(active_customers)]

# Merge with normalized data
merged_2021 = pd.merge(filtered_yearly_data[filtered_yearly_data['CALENDAR_YEAR'] == 2021],
                       normalized_data, on='CUSTOMER_ID', how='left')
merged_2022 = pd.merge(filtered_yearly_data[filtered_yearly_data['CALENDAR_YEAR'] == 2022],
                       normalized_data, on='CUSTOMER_ID', how='left')

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize features for both 2021 and 2022 datasets
features_to_normalize = ['NUM_TRANSACTIONS', 'TOTAL_SPEND']
for feature in features_to_normalize:
    merged_2021[feature] = scaler.fit_transform(merged_2021[[feature]])
    merged_2022[feature] = scaler.transform(merged_2022[[feature]])

# Define functions to calculate CV and CLS
def calculate_customer_value(row):
    return row['N_TotalRevenue']

def estimate_customer_lifespan_with_coefficients(row, recency_weight, frequency_weight):
    return (recency_weight * row['N_FirstRecency']) + (frequency_weight * row['N_YearlyFrequency'])

# Extract weights from your regression model
regression_weights = {
    'N_FirstRecency': 1.606693e+09,
    'N_YearlyFrequency': 7.039319e-04
}

# Calculate CV, CLS, and CLV for merged_2021 and merged_2022
merged_2021['CV'] = merged_2021.apply(calculate_customer_value, axis=1)
merged_2021['CLS'] = merged_2021.apply(lambda row: estimate_customer_lifespan_with_coefficients(row,
                                         regression_weights['N_FirstRecency'],
                                         regression_weights['N_YearlyFrequency']), axis=1)
merged_2021['CLV'] = merged_2021['CV'] * merged_2021['CLS']

merged_2022['CV'] = merged_2022.apply(calculate_customer_value, axis=1)
merged_2022['CLS'] = merged_2022.apply(lambda row: estimate_customer_lifespan_with_coefficients(row,
                                         regression_weights['N_FirstRecency'],
                                         regression_weights['N_YearlyFrequency']), axis=1)
merged_2022['CLV'] = merged_2022['CV'] * merged_2022['CLS']

# Calculate the high frequency threshold
high_frequency_threshold = normalized_data['N_YearlyFrequency'].quantile(0.75)

# Define thresholds for segmentation
high_clv_threshold = merged_2021['CLV'].quantile(0.75)
long_cls_threshold = merged_2021['CLS'].quantile(0.75)

# Function for customer segmentation with frequency
def categorize_customer_with_frequency(row, clv_threshold, cls_threshold, freq_threshold):
    frequency_segment = 'High Frequency' if row['N_YearlyFrequency'] >= freq_threshold else 'Low Frequency'

    if row['CLV'] >= clv_threshold and row['CLS'] >= cls_threshold:
        return f'High Value - Long Lifespan - {frequency_segment}'
    elif row['CLV'] >= clv_threshold and row['CLS'] < cls_threshold:
        return f'High Value - Short Lifespan - {frequency_segment}'
    elif row['CLV'] < clv_threshold and row['CLS'] >= cls_threshold:
        return f'Low Value - Long Lifespan - {frequency_segment}'
    else:
        return f'Low Value - Short Lifespan - {frequency_segment}'

# Apply segmentation with frequency
merged_2021['Customer_Segment'] = merged_2021.apply(lambda row: categorize_customer_with_frequency(row,
                                                                                                   high_clv_threshold,
                                                                                                   long_cls_threshold,
                                                                                                   high_frequency_threshold), axis=1)
merged_2022['Customer_Segment'] = merged_2022.apply(lambda row: categorize_customer_with_frequency(row,
                                                                                                   high_clv_threshold,
                                                                                                   long_cls_threshold,
                                                                                                   high_frequency_threshold), axis=1)

# Validation
errors = merged_2022['TOTAL_SPEND'] - merged_2021['CLV']
mean_error = errors.mean()
median_error = errors.median()
percentile_25_error = errors.quantile(0.25)
percentile_50_error = errors.quantile(0.50)
percentile_75_error = errors.quantile(0.75)
positive_errors = (errors > 0).sum()
negative_errors = (errors < 0).sum()

# Print validation metrics
print("Mean Error:", mean_error)
print("Median Error:", median_error)
print("25th Percentile of Error:", percentile_25_error)
print("50th Percentile of Error:", percentile_50_error)
print("75th Percentile of Error:", percentile_75_error)
print("Number of Positive Errors (Underestimation):", positive_errors)
print("Number of Negative Errors (Overestimation):", negative_errors)


Mean Error: -23808783.22409544
Median Error: -12822339.505619768
25th Percentile of Error: -29050265.4783621
50th Percentile of Error: -12822339.505619768
75th Percentile of Error: -7105688.768388426
Number of Positive Errors (Underestimation): 0
Number of Negative Errors (Overestimation): 70442


#### 2.3 Testing the Revised Model for CLS Calculation without Coefficients and Focusing on Customer Activity in 2021

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the original and normalized datasets
original_data = pd.read_csv('COLUMBIA_CAPSTONE_1M_FINAL_DATASET.csv', sep="|", header=None)
normalized_data = pd.read_csv('/content/hotel_cluster_normalized.csv')
original_data.columns = ['CUSTOMER_ID', 'CALENDAR_YEAR', 'CALENDAR_MONTH', 'PRODUCT_CATEGORY',
                         'LATEST_TRANSACTION_DATE', 'NUM_TRANSACTIONS', 'TOTAL_SPEND', 'MARRIOTT_REVENUE']

# Aggregate the original data by customer and year
yearly_data = original_data.groupby(['CUSTOMER_ID', 'CALENDAR_YEAR']).agg({
    'NUM_TRANSACTIONS': 'sum',
    'TOTAL_SPEND': 'sum'
}).reset_index()

# Identify customers with transactions in 2021 and before 2021
customers_2021 = set(yearly_data[yearly_data['CALENDAR_YEAR'] == 2021]['CUSTOMER_ID'])
customers_before_2021 = set(yearly_data[yearly_data['CALENDAR_YEAR'] < 2021]['CUSTOMER_ID'])
active_customers = customers_2021.intersection(customers_before_2021)

# Filter the yearly_data to include only these customers
filtered_yearly_data = yearly_data[yearly_data['CUSTOMER_ID'].isin(active_customers)]

# Merge with normalized data
merged_2021 = pd.merge(filtered_yearly_data[filtered_yearly_data['CALENDAR_YEAR'] == 2021],
                       normalized_data, on='CUSTOMER_ID', how='left')
merged_2022 = pd.merge(filtered_yearly_data[filtered_yearly_data['CALENDAR_YEAR'] == 2022],
                       normalized_data, on='CUSTOMER_ID', how='left')

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize features for both 2021 and 2022 datasets
features_to_normalize = ['NUM_TRANSACTIONS', 'TOTAL_SPEND']
for feature in features_to_normalize:
    merged_2021[feature] = scaler.fit_transform(merged_2021[[feature]])
    merged_2022[feature] = scaler.transform(merged_2022[[feature]])

# Calculate CV, CLS, and CLV for merged_2021 and merged_2022
merged_2021['CV'] = merged_2021['N_TotalRevenue']
merged_2021['CLS'] = merged_2021['N_FirstRecency'] * merged_2021['N_YearlyFrequency']
merged_2021['CLV'] = merged_2021['CV'] * merged_2021['CLS']

merged_2022['CV'] = merged_2022['N_TotalRevenue']
merged_2022['CLS'] = merged_2022['N_FirstRecency'] * merged_2022['N_YearlyFrequency']
merged_2022['CLV'] = merged_2022['CV'] * merged_2022['CLS']

# Calculate the high frequency threshold
high_frequency_threshold = normalized_data['N_YearlyFrequency'].quantile(0.75)

# Define thresholds for segmentation
high_clv_threshold = merged_2021['CLV'].quantile(0.75)
long_cls_threshold = merged_2021['CLS'].quantile(0.75)

# Function for customer segmentation with frequency
def categorize_customer_with_frequency(row, clv_threshold, cls_threshold, freq_threshold):
    frequency_segment = 'High Frequency' if row['N_YearlyFrequency'] >= freq_threshold else 'Low Frequency'

    if row['CLV'] >= clv_threshold and row['CLS'] >= cls_threshold:
        return f'High Value - Long Lifespan - {frequency_segment}'
    elif row['CLV'] >= clv_threshold and row['CLS'] < cls_threshold:
        return f'High Value - Short Lifespan - {frequency_segment}'
    elif row['CLV'] < clv_threshold and row['CLS'] >= cls_threshold:
        return f'Low Value - Long Lifespan - {frequency_segment}'
    else:
        return f'Low Value - Short Lifespan - {frequency_segment}'

# Apply segmentation with frequency
merged_2021['Customer_Segment'] = merged_2021.apply(lambda row: categorize_customer_with_frequency(row,
                                                                                                   high_clv_threshold,
                                                                                                   long_cls_threshold,
                                                                                                   high_frequency_threshold), axis=1)
merged_2022['Customer_Segment'] = merged_2022.apply(lambda row: categorize_customer_with_frequency(row,
                                                                                                   high_clv_threshold,
                                                                                                   long_cls_threshold,
                                                                                                   high_frequency_threshold), axis=1)

# Validation
errors = merged_2022['TOTAL_SPEND'] - merged_2021['CLV']
mean_error = errors.mean()
median_error = errors.median()
percentile_25_error = errors.quantile(0.25)
percentile_50_error = errors.quantile(0.50)
percentile_75_error = errors.quantile(0.75)
positive_errors = (errors > 0).sum()
negative_errors = (errors < 0).sum()

# Print validation metrics
print("Mean Error:", mean_error)
print("Median Error:", median_error)
print("25th Percentile of Error:", percentile_25_error)
print("50th Percentile of Error:", percentile_50_error)
print("75th Percentile of Error:", percentile_75_error)
print("Number of Positive Errors (Underestimation):", positive_errors)
print("Number of Negative Errors (Overestimation):", negative_errors)


Mean Error: 0.0058793695168682025
Median Error: 0.004850446257223948
25th Percentile of Error: 0.004693186299018631
50th Percentile of Error: 0.004850446257223948
75th Percentile of Error: 0.005530977674585992
Number of Positive Errors (Underestimation): 69695
Number of Negative Errors (Overestimation): 747


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Filter only those customers who are present in both 2021 and 2022 data
common_customers = set(merged_2021['CUSTOMER_ID']).intersection(set(merged_2022['CUSTOMER_ID']))

# Filter both datasets to include only these common customers
merged_2021_common = merged_2021[merged_2021['CUSTOMER_ID'].isin(common_customers)]
merged_2022_common = merged_2022[merged_2022['CUSTOMER_ID'].isin(common_customers)]

# Ensure the data is aligned by sorting based on CUSTOMER_ID
merged_2021_common.sort_values(by='CUSTOMER_ID', inplace=True)
merged_2022_common.sort_values(by='CUSTOMER_ID', inplace=True)

# Predicting 2022 Total Spend based on 2021 data
X = merged_2021_common[['CV', 'CLS']]
y = merged_2022_common['TOTAL_SPEND']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate RMSE and R^2
rmse = mean_squared_error(y_test, y_pred, squared=False)
r_squared = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2:", r_squared)


RMSE: 0.003570089517172678
R^2: 0.2638306292622018


<ipython-input-30-63d6f2747833>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_2021_common.sort_values(by='CUSTOMER_ID', inplace=True)


The validation results indicate the following:

Mean Error: The average difference between predicted CLV for 2021 and actual total spend in 2022 is approximately 0.0059. Since this is a small positive number, it suggests a slight overall underestimation.

Median Error: The median error is also positive, around 0.0048, reinforcing the trend of underestimation.

Percentile Errors: The 25th, 50th (median), and 75th percentile errors all align with this pattern of slight underestimation.

Positive Errors (Underestimation): The majority of errors (69695 cases) are positive, meaning the model tends to underestimate the total spend.

Negative Errors (Overestimation): A smaller number of cases (747) where the model overestimates the total spend.


In [ ]:
merged_2021.head()

,CUSTOMER_ID,CALENDAR_YEAR,NUM_TRANSACTIONS,TOTAL_SPEND,N_Recency,N_Frequency,N_Monetary,N_FirstRecency,N_LifetimeRecency,N_YearlyFrequency,...,Segment_No Ancillary Spenders,Revenue_Segment_High Ancillary Spenders,Revenue_Segment_Low Ancillary Spenders,Revenue_Segment_Moderate Ancillary Spenders,Revenue_Segment_No Ancillary Spenders,Cluster,CV,CLS,CLV,Customer_Segment
0,3431,2021,0.000758,0.004722,0.948892,0.000752,0.004263,0.291918,0.240841,0.001454,...,0,0,1,0,0,2,0.026720,0.000425,0.000011,Low Value - Short Lifespan - High Frequency
1,333339,2021,0.009099,0.011231,0.995972,0.008649,0.015008,0.500755,0.496790,0.017520,...,0,0,0,1,0,0,0.046442,0.008773,0.000407,High Value - Long Lifespan - High Frequency
2,353130,2021,0.006571,0.005258,0.996349,0.006643,0.006504,0.930514,0.926980,0.013975,...,0,0,1,0,0,2,0.032179,0.013004,0.000418,High Value - Long Lifespan - High Frequency
3,353730,2021,0.049918,0.007090,0.995972,0.052958,0.011403,0.931898,0.927987,0.046252,...,0,0,1,0,0,4,0.035184,0.043103,0.001517,High Value - Long Lifespan - High Frequency
4,353832,2021,0.001264,0.004743,0.956445,0.001253,0.006668,0.997231,0.953796,0.008415,...,0,0,1,0,0,2,0.042502,0.008391,0.000357,High Value - Long Lifespan - High Frequency


### 3. Analyze the overall customer segments
##### Aggregate Data for All Years: Instead of filtering data for specific years, I'll aggregate and analyze the data for all years available in the dataset.

##### Calculate CLV for the Entire Dataset: Perform the calculations of CV, CLS, and CLV for the entire dataset.

##### Segment Customers Based on the Entire Dataset: Apply the segmentation logic to the entire dataset.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the original and normalized datasets
original_data = pd.read_csv('COLUMBIA_CAPSTONE_1M_FINAL_DATASET.csv', sep="|", header=None)
normalized_data = pd.read_csv('/content/hotel_cluster_normalized.csv')
original_data.columns = ['CUSTOMER_ID', 'CALENDAR_YEAR', 'CALENDAR_MONTH', 'PRODUCT_CATEGORY',
                         'LATEST_TRANSACTION_DATE', 'NUM_TRANSACTIONS', 'TOTAL_SPEND', 'MARRIOTT_REVENUE']

# Aggregate the original data by customer for all years
yearly_data = original_data.groupby(['CUSTOMER_ID']).agg({
    'NUM_TRANSACTIONS': 'sum',
    'TOTAL_SPEND': 'sum'
}).reset_index()

# Merge with normalized data
merged_data = pd.merge(yearly_data, normalized_data, on='CUSTOMER_ID', how='left')

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize features
features_to_normalize = ['NUM_TRANSACTIONS', 'TOTAL_SPEND']
for feature in features_to_normalize:
    merged_data[feature] = scaler.fit_transform(merged_data[[feature]])

# Calculate CV, CLS, and CLV
merged_data['CV'] = merged_data['N_TotalRevenue']
merged_data['CLS'] = merged_data['N_FirstRecency'] * merged_data['N_YearlyFrequency']
merged_data['CLV'] = merged_data['CV'] * merged_data['CLS']

# Calculate the high frequency threshold
high_frequency_threshold = normalized_data['N_YearlyFrequency'].quantile(0.75)

# Define thresholds for segmentation
high_clv_threshold = merged_data['CLV'].quantile(0.75)
long_cls_threshold = merged_data['CLS'].quantile(0.75)

# Function for customer segmentation with frequency
def categorize_customer_with_frequency(row, clv_threshold, cls_threshold, freq_threshold):
    frequency_segment = 'High Frequency' if row['N_YearlyFrequency'] >= freq_threshold else 'Low Frequency'

    if row['CLV'] >= clv_threshold and row['CLS'] >= cls_threshold:
        return f'High Value - Long Lifespan - {frequency_segment}'
    elif row['CLV'] >= clv_threshold and row['CLS'] < cls_threshold:
        return f'High Value - Short Lifespan - {frequency_segment}'
    elif row['CLV'] < clv_threshold and row['CLS'] >= cls_threshold:
        return f'Low Value - Long Lifespan - {frequency_segment}'
    else:
        return f'Low Value - Short Lifespan - {frequency_segment}'

# Apply segmentation with frequency
merged_data['Customer_Segment'] = merged_data.apply(lambda row: categorize_customer_with_frequency(row,
                                                                                                   high_clv_threshold,
                                                                                                   long_cls_threshold,
                                                                                                   high_frequency_threshold), axis=1)

# Explore the segments
segment_counts = merged_data['Customer_Segment'].value_counts()
print(segment_counts)


Low Value - Short Lifespan - Low Frequency      652203
High Value - Long Lifespan - High Frequency     152568
Low Value - Short Lifespan - High Frequency      96973
High Value - Long Lifespan - Low Frequency       96609
High Value - Short Lifespan - Low Frequency        653
Low Value - Long Lifespan - Low Frequency          534
Low Value - Long Lifespan - High Frequency         290
High Value - Short Lifespan - High Frequency       170
Name: Customer_Segment, dtype: int64
